# Hybrid Approach -- Scenario I

In this notebook, we report the code related to the *hybrid* appraoch in our paper [[1]] (#ourpaper). We consider the delay caused by the feedback loop used by the mobile user equipment to send to the base station the channel state information (CSI). Due to this feedback delay, the CSI available at the base station becomes outdated. In our paper, we propose a hybrid *data-driven* and *model-based* approach. The model-based part consists of using a FIR Wiener filter to predict the instantaneous CSI given the channel history, which we assume available at the base station. The data-driven part consists of a neural network whose input is the output of the Wiener filter. Ths network maps the CSI to the probability of an error event for all the MCSs and selects the MCS that maximizes the spectral efficiency

Under Scenario I, we implement a different Wiener filter and we train a differnt neural network for each combination of delay, doppler, and signal-to-noise ratio.

It must be noted that the training **datasets listed below in the code are not in the repository** due to space limitations. At any rate, in the reposiroty the reader can find the "Generate_Data.ipnyb" which can be run to generate the datasets for training. A link to an external storage with the datsets for training will be provided soon.

**Note**: the training might take some hours, depending on the available computational resources, the dimension of the training set, the dimension of the network, and the number of epochs. 


<a id='ourpaper'></a> [1] "Wireless link adaptation - a hybrid data-driven and model-based approach", Lissy Pellaco, Vidit Saxena, Mats Bengtsson, Joakim Jaldén. Submitted to WCNC 2020.

## Import libraries and utility functions

In [ ]:
import numpy as np
import time
from keras.optimizers import Adam
import utilities as utils
from keras.backend.tensorflow_backend import set_session
from keras.backend import clear_session
import tensorflow as tf

In [ ]:
# Flag used to indicate if the channel is noisy
CHANNEL_EST_NOISE = True
# Number of subcarriers in the OFDM
NROF_SUBCARRIERS = 72
# Number of MCSs
NROF_MCS = 29
# Number of Wiener filter taps
Wiener_filter_taps=10
# Parameters related to neural network training
BATCH_SIZE = 32
NROF_EPOCHS = 10
TRAINING_FRACTION = 1

# Flag to indicate if the trained models should be saved
save_model = False

## Load the Dataset

The channel dataset is a dict with the following keys :  
 - 'channel'
     - Complex channel coefficients 
     - Numpy array [ NROF_FRAMES x NROF_SUBCARRIERS x NROF_SNRS]
 - 'block_success'
      - Binary success events (ACKs)
      - Numpy array [ NROF_FRAMES x NROF_MCS x NROF_SNRS]
 - 'snrs_db '      
     - Evaluated average SNR values
     - Numpy array [ NROF_SNRS ]
 - 'block_sizes'
     - Evaluated transport block sizes
     - Numpy array [ NROF_MCS ]
     
The name of the dataset, e.g., ITU_VEHICULAR_B_1000_210_111_72_5dB, is to be interpreted in this way: 
 - channel model (ITU_VEHICULAR_B)
 - number of channel realizations per batch (1000)
 - number of batches of the dataset (210)
 - doppler in Hz cast to integer (111)
 - number of subcarriers (72)
 - snr (5 dB)
 
It must be noted that the **training datasets listed below in the code are not in the repository due to space limitations**. At any rate, in the repository, the reader can find the "radio_data/Generate_Data.ipnyb" which can be run to generate the datasets for training.

In [ ]:
# The files stored in the file_set ARE NOT in the repository due to space limitations, but the reader has access to the
# "source/Generate_Data.ipynb" which we used to generate the training datasets

file_set=[
            'datasets/ITU_VEHICULAR_B_1000_210_16.67_72_5dB.npy',
          'datasets/ITU_VEHICULAR_B_1000_210_16.67_72_15dB.npy',
          'datasets/ITU_VEHICULAR_B_1000_500_16.67_72_25dB.npy',
        ]
# Doppler frequency related to the datasets in "file_set"
dopplers_set = (20.0 / 3.0) * np.array([16.67,16.67,16.67])

# Snrs related to the datasets in "file_set" 
snrs_set=[5,15,25]

# Number of batches related to the datasets in "file_set"
num_batches_per_dataset=[210,210,500]

# Maximum feedback delay that we consider
maximum_delay=9

## Build the neural network model, define the optimizer, and the cost function

In [ ]:
def create_ann_model():
    from keras.models import Sequential
    from keras.layers import Dense, Dropout

    model = Sequential()
    model.add( Dense( 1024, 
                      input_dim = NROF_SUBCARRIERS * 2, 
                      kernel_initializer='normal', 
                      activation='relu' ) )

    model.add( Dense( 512,
                      kernel_initializer = 'normal', 
                      activation='relu' ) )
        
    model.add( Dense( 1024, 
                      kernel_initializer = 'normal', 
                      activation='relu' ) )
    
    model.add( Dense( NROF_MCS, 
                      kernel_initializer='normal', 
                      activation='sigmoid' ) )

    # Compile model
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])  # for binary classification
 
    return model

# FIR Wiener filter

In [ ]:
def apply_wiener_filtering( freq_domain_channel, snrs_dB, doppler_freq, delay = 1, N = 10 ):
    nrof_samples, nrof_subcarriers= freq_domain_channel.shape
    
    filtered_channel_freq_response = np.ndarray( freq_domain_channel.shape, dtype = np.complex128 )
   
      
    
    snr = 10 ** ( 0.1 *snrs_dB )

    channel_sampling_interval = 0.001
    autocorrelation_of_reference = utils.autocorrelation( np.arange(0,N,1),
                                                              doppler_freq,
                                                              channel_sampling_interval )

    crosscorrelation = utils.autocorrelation( np.arange(delay, N + delay, 1),
                                                  doppler_freq,
                                                  channel_sampling_interval )

    Wiener_coeff = utils.Wiener_filter_coeff_scaled( autocorrelation_of_reference,
                                                         crosscorrelation,
                                                         delay,
                                                         N,
                                                         snr,
                                                         True,
                                                         doppler_freq,
                                                         channel_sampling_interval )

    for subc_index in range( nrof_subcarriers ):
            
        subcarrier_response = freq_domain_channel[:, subc_index]
        
            #Apply the Wiener filter
        filtered_subc_response = np.convolve( Wiener_coeff, subcarrier_response, "full")
        filtered_channel_freq_response[ :, subc_index] = filtered_subc_response[ : -N + 1 ]
            
    return filtered_channel_freq_response




## Train the Neural Network over Wiener filter's output

In [ ]:
start_time = time.time()

#loop over the delays
for DELAY in range(0,maximum_delay +1):
    
    #loop over the snrs
    for i in range(0,len(snrs_set)):
        
        DOPPLER=dopplers_set[i]
        FILE=file_set[i]
        SELECTED_SNR=snrs_set[i]
        DATASET = np.load( FILE, allow_pickle = True )[()]
        
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True  

        sess = tf.Session( config = config )
        set_session(sess) 
        model = create_ann_model( )

        
        for j in range(0,num_batches_per_dataset[i]):
            channel_coeff  = []
            block_success  = []

            DATASET_channel=DATASET['channel'][:,:,j]
            DATASET_block_success=DATASET['block_success'][:,:,j]

            nrof_train_samples = int( TRAINING_FRACTION * DATASET_channel.shape[0] )


            coeff = utils.calculate_channel_coefficients_scaled_fixed_snr( DATASET_channel[ :nrof_train_samples, :],
                                                                         DATASET['snrs_db'][0],
                                                                         channel_estimation_noise = CHANNEL_EST_NOISE )
            
            coeff_filtered = apply_wiener_filtering( coeff, 
                                                     DATASET[ 'snrs_db' ][0], 
                                                     doppler_freq = DOPPLER, 
                                                     delay = DELAY, 
                                                     N = Wiener_filter_taps )
            
            
            
            channel_coeff.append( coeff_filtered )
            block_success.append( DATASET_block_success[ :nrof_train_samples, :] )

            channel_coeff = np.vstack( channel_coeff )
            block_success = np.vstack( block_success )

            NROF_FRAMES, _ = channel_coeff.shape


            channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )
            
            
            if DELAY > 0:
                train_input  =( channel_coeff_concat[ :-DELAY, : ] )
            else:
                train_input  =( channel_coeff_concat[ :, :]  )

            train_target = ( block_success [ DELAY :, :] )
           

            train_input, train_target = utils.shuffle_data( train_input, train_target )

            history = model.fit( train_input, 
                     train_target, 
                     batch_size = BATCH_SIZE, 
                     epochs     = NROF_EPOCHS, 
                     validation_split = 0.1, 
                     verbose    = 0 ) # the "verbose parameter" can be changed to display more about the training progess of each epoch
            
        file_to_save = 'Trained_models_ScenarioI/ANN_MCS_PRED_WIENER_DELAY_%d_DP_%d_SNR_%d.h5'%(DELAY,DOPPLER,SELECTED_SNR)
        if save_model == True:
            model.save( file_to_save )
        clear_session()
        print("--- %s seconds ---" % (time.time() - start_time))
        